In [1]:
from datamart.augment import Augment
from datamart.utilities.utils import Utils
import pandas as pd
import json

In [2]:
es_index = "datamart_all"

augment = Augment(es_index=es_index)

#### Read the supply dataset

In [3]:
old_df = pd.read_csv("./example/fifa_example/fifa.csv")
print(old_df.iloc[:10, :])

   d3mIndex  GameID        Date          Team      Opponent  \
0         0       0  14-06-2018        Russia  Saudi Arabia   
1         1       1  14-06-2018  Saudi Arabia        Russia   
2         2       2  15-06-2018         Egypt       Uruguay   
3         3       3  15-06-2018       Uruguay         Egypt   
4         4       4  15-06-2018       Morocco          Iran   
5         5       5  15-06-2018          Iran       Morocco   
6         6       6  15-06-2018      Portugal         Spain   
7         7       7  15-06-2018         Spain      Portugal   
8         8       8  16-06-2018        France     Australia   
9         9       9  16-06-2018     Australia        France   

   Ball Possession %  Off-Target  Blocked  Offsides  Saves  Pass Accuracy %  \
0                 40           3        3         3      0               78   
1                 60           3        3         1      2               86   
2                 43           3        2         1      3           

#### Datamart finds named entity column for query

In [4]:
for col in old_df:
    if Utils.is_column_able_to_query(old_df[col]):
        print(col)

Team
Opponent
Round
PSO


#### Query by named entity column and specified string

In [5]:
query_string = "fifa"

query_result = augment.query(
    col=old_df['Team'],
    query_string=query_string
)
print("Team column matched:", len(query_result))


Team column matched: 1


In [6]:
print([x["_source"]["datamart_id"] for x in query_result])

[127860000]


#### Show the metadata hitted

In [7]:
print(json.dumps(query_result[0], indent=2))

{
  "_index": "datamart_all",
  "_type": "_doc",
  "_id": "127860000",
  "_score": 32.552834,
  "_source": {
    "datamart_id": 127860000,
    "title": "FIFA World Cup",
    "description": "FIFA World Cup",
    "url": "https://www.football-data.org",
    "keywords": [
      "football",
      "competition"
    ],
    "provenance": {
      "source": "www.football-data.org"
    },
    "materialization": {
      "python_path": "football_match_materializer",
      "arguments": {
        "uri": "/v2/competitions/2000/matches?limit=999",
        "token": "d019bc4541c9490fabcba6806cbcc42b"
      }
    },
    "variables": [
      {
        "datamart_id": 127860001,
        "name": "id",
        "semantic_type": [
          "http://schema.org/Integer"
        ],
        "description": "column name: id, dtype: int64"
      },
      {
        "datamart_id": 127860002,
        "name": "season_id",
        "semantic_type": [
          "http://schema.org/Integer"
        ],
        "description": "co

#### Materialize the dataset and show some lines

In [8]:
new_df = Utils.get_dataset(metadata=query_result[0]["_source"])
print(new_df.iloc[:10, :])

       id  season_id season_startDate season_endDate  season_currentMatchday  \
0  200000          1       2018-06-14     2018-07-15                       3   
1  200001          1       2018-06-14     2018-07-15                       3   
2  200006          1       2018-06-14     2018-07-15                       3   
3  200007          1       2018-06-14     2018-07-15                       3   
4  200012          1       2018-06-14     2018-07-15                       3   
5  200018          1       2018-06-14     2018-07-15                       3   
6  200013          1       2018-06-14     2018-07-15                       3   
7  200019          1       2018-06-14     2018-07-15                       3   
8  200024          1       2018-06-14     2018-07-15                       3   
9  200030          1       2018-06-14     2018-07-15                       3   

                utcDate    status  matchday        stage    group  \
0  2018-06-14T15:00:00Z  FINISHED       1.0  GROUP

#### Columns useful for join

In [9]:
print(old_df[["Date", "Team", "Opponent"]].iloc[:10, :])

         Date          Team      Opponent
0  14-06-2018        Russia  Saudi Arabia
1  14-06-2018  Saudi Arabia        Russia
2  15-06-2018         Egypt       Uruguay
3  15-06-2018       Uruguay         Egypt
4  15-06-2018       Morocco          Iran
5  15-06-2018          Iran       Morocco
6  15-06-2018      Portugal         Spain
7  15-06-2018         Spain      Portugal
8  16-06-2018        France     Australia
9  16-06-2018     Australia        France


In [10]:
print(new_df[["utcDate", "homeTeam_name", "awayTeam_name"]].iloc[:10, :])

                utcDate homeTeam_name awayTeam_name
0  2018-06-14T15:00:00Z        Russia  Saudi Arabia
1  2018-06-15T12:00:00Z         Egypt       Uruguay
2  2018-06-15T15:00:00Z       Morocco          Iran
3  2018-06-15T18:00:00Z      Portugal         Spain
4  2018-06-16T10:00:00Z        France     Australia
5  2018-06-16T13:00:00Z     Argentina       Iceland
6  2018-06-16T16:00:00Z          Peru       Denmark
7  2018-06-16T19:00:00Z       Croatia       Nigeria
8  2018-06-17T12:00:00Z    Costa Rica        Serbia
9  2018-06-17T15:00:00Z       Germany        Mexico


#### Default join for current datamart

In [11]:
df = augment.join(left_df=old_df,
                  right_df=new_df,
                  left_columns=[[old_df.columns.tolist().index(x)] for x in ["Team", "Opponent"]],
                  right_columns=[[new_df.columns.tolist().index(x)] for x in ["homeTeam_name", "awayTeam_name"]])

df.to_csv("./example/fifa_example/augmented.csv", index=False)

### Because of the supplied dataset has two rows for one single game.
### Current simple default join can not augment both of them. 
### ISI is working on some sophasticate join which should be able to handle cases not limited to this

In [ ]:
print("\n - try to join:")
joined_df = augment.join(
    left_df=old_df,
    right_df=new_df,
    left_columns=[[old_df.columns.tolist().index(x)] for x in ["Team", "Opponent"]],
    right_columns=[[new_df.columns.tolist().index(x)] for x in ["homeTeam_name", "awayTeam_name"]],      
    left_metadata=None,
    right_metadata=query_result[0]["_source"],
    joiner="rltk"
)
print(joined_df)
